install if your environment does not have cbsyst

In [ ]:
# pip install cbsyst

In [ ]:
import numpy as np
import pandas as pd
import os
import zipfile
import urllib.request as ureq
from tqdm import tqdm
import cbsyst as cb
import xarray as xr
import gcsfs
import cmocean as cm
import matplotlib.pyplot as plt
import requests
import re

In [ ]:
# Define the surface depth of the data
surface_depth = 10


In [ ]:
# From: https://glodap.info/index.php/merged-and-adjusted-data-product-v2-2023/
# Copy the link of "Merged Master File (ZIPped CSV)"

# Download path
glodap_url ='https://glodap.info/glodap_files/v2.2023/GLODAPv2.2023_Merged_Master_File.csv.zip'
# GCS path to save the data
save_path = 'YOUR_GCS_PATH/Taylor_data/databases'
# Edit the file name if needed.
gcs_path = f'{save_path}/raw/GLODAPv2.2023_Merged_Master_File.csv.zip'

# If you want to change the outcome .nc save path or file name, search for "zarr_gcs_path" later in this notebook


In [ ]:
# Download and save to GCS
fs = gcsfs.GCSFileSystem()

# download and save to GCS
if fs.exists(gcs_path):
    print(f"File already exists at {gcs_path}. Skipping download.")
else:
    response = requests.get(glodap_url, stream=True)
    if response.status_code == 200:
        with fs.open(gcs_path, 'wb') as gcs_file:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:  # filter out keep-alive new chunks
                    gcs_file.write(chunk)
        print(f"Succesfully downloaded and saved to {gcs_path}")
    else:
        print(f"Error! Could not download: {response.status_code}")

In [ ]:
fs = gcsfs.GCSFileSystem()
fs.ls('YOUR_GCS_PATH/Taylor_data/databases/raw')

In [ ]:
# load data from GCS
# zip to csv
zf = zipfile.ZipFile(fs.open(gcs_path))

gd = pd.read_csv(zf.open('GLODAPv2.2023_Merged_Master_File.csv'))

In [ ]:
# Remove "G2" of all column names. Could be manually set if changes apply in the future
gd.columns = gd.columns.str.replace('G2', '')

gd.head()

In [ ]:
# replace missing values(-9999) with nan
gd.replace(-9999, np.nan, inplace=True)

# isolate good data only (flag = 2)
# From https://essd.copernicus.org/articles/12/3653/2020/ 
# From the notes of table 1 and 2, we know that those column names end with "f" represent the WOCE flags and the flag = 2 means the data is acceptable.
gd.loc[gd.phtsinsitutpf != 2, 'phtsinsitutp'] = np.nan
gd.loc[gd.tco2f != 2, 'tco2'] = np.nan
gd.loc[gd.talkf != 2, 'talk'] = np.nan
gd.loc[gd.salinityf != 2, 'salinity'] = np.nan
gd.loc[gd.phosphatef != 2, 'phosphate'] = np.nan
gd.loc[gd.silicatef != 2, 'silicate'] = np.nan

gd.dropna(subset=['phtsinsitutp', 'tco2', 'talk', 'temperature', 'salinity', 'pressure', 'silicate', 'phosphate'], inplace=True)


In [ ]:
# to check what unique values are there in 'salinityf' column. Should only have "2". 
gd.salinityf.unique()

In [ ]:
# Put year/month/day/hour into sensible format
gd['date'] = pd.to_datetime(gd[['year', 'month', 'day']])

### Convert from -180 / 180 to 0/360 longitude
def f(row):
    if row['longitude'] < 0:
        val = row['longitude'] + 360
    else:
        val = row['longitude']
    return val

### Apply function to dataframe 
gd['lon_0360'] = gd.apply(f, axis=1)

### convert pressure to bar
gd.pressure /= 10 

### Calculate pCO2 using cbsyst
calc_pco2 = cb.Csys(pHtot=gd.phtsinsitutp, DIC=gd.tco2, T_in=gd.temperature, S_in=gd.salinity, P_in=gd.pressure, PT=gd.phosphate, SiT=gd.silicate, BT=415.7)

### put pCO2 into GLODAP dataframe - gd
gd['pco2'] = calc_pco2.pCO2

### Groupby profile and take index where depth is min
gd = gd.loc[gd.groupby(['cruise','station','cast'])['depth'].idxmin()]

### Set index to dates
gd_sub = gd.loc[:, ['date', 'latitude', 'lon_0360', 'depth', 'pco2']].rename(columns={'latitude':'lat', 'lon_0360':'lon', 'pco2':'spco2', 'date':'time'}).set_index(['time'])

### Only look in top 'surface_depth' meters, here we use 10m for now
# Depth is defined earlier, search for "surface_depth" if you want to change it
gd_sub = gd_sub.where(gd_sub['depth']<=surface_depth).dropna()


In [ ]:
gd_sub.tail()

In [ ]:
## Takes a flat array of ungridded data and grids it to the grid of the given xr.DataArray

# Date range to create monthly mask over
# list of start and stop dates
# Auto detect the date range. But could be manually set.
min_date_str = gd_sub.index.min().strftime('%Y-%m')
max_date_str = gd_sub.index.max().strftime('%Y-%m')
date_range = [min_date_str,max_date_str]
# date_range = ['1982-01', '2021-12']
print(f'Date range is:{date_range[0]} to {date_range[1]}.')

# Define spatial range
lon = np.arange(0.5, 360, 1)
lat = np.arange(-89.5, 90, 1)
time = pd.date_range(start=f'{date_range[0]}-01T00:00:00.000000000',
                     end=f'{date_range[1]}-01T00:00:00.000000000',
                     freq='MS') + np.timedelta64(14, 'D')

# Create DataArray
xda = xr.DataArray(np.zeros((len(time), len(lat), len(lon))),
                   dims=['time', 'lat', 'lon'],
                   coords=[time, lat, lon])

# Convert data into series
ds_gd = pd.DataFrame({
    'time': gd_sub.index,
    'lon': gd_sub.lon.values,
    'lat': gd_sub.lat.values,
    'spco2': gd_sub.spco2.values
})


In [ ]:
ds_gd.head()

In [ ]:
# Check if coordinates are present and correctly formatted
for key in ['time', 'lat', 'lon']:
     assert key in xda.dims, '`{}` is not in the input DataArray'.format(key)
assert all(np.diff(xda.time.values) > np.timedelta64(0, 'D')), 'time is not strictly increasing'
assert all(np.diff(xda.lat.values) > 0), 'latitude is not strictly increasing'
assert all(np.diff(xda.lon.values) > 0), 'longitude is not strictly increasing'
assert np.issubdtype(ds_gd['time'].dtype, np.datetime64), 'time must be in datetime64 format'

# Create bins for time, latitude, and longitude
def make_bins(dim):
    delta = np.diff(dim) / 2
    bins = np.r_[dim[0] - delta[0], dim[1:] - delta, dim[-1] + delta[-1]]
    return bins

tbins = make_bins(xda.time.values)
ybins = make_bins(xda.lat.values)
xbins = make_bins(xda.lon.values)

# Bin the data and get indices
tidx = pd.cut(ds_gd['time'], tbins, labels=np.arange(len(xda.time)))
yidx = pd.cut(ds_gd['lat'], ybins, labels=np.arange(len(xda.lat)))
xidx = pd.cut(ds_gd['lon'], xbins, labels=np.arange(len(xda.lon)))

print("Number of NaNs in bin indices:")
print(f"tidx: {tidx.isna().sum()}, yidx: {yidx.isna().sum()}, xidx: {xidx.isna().sum()}")

# Group by bins and calculate mean and standard deviation
df = (pd.DataFrame({'spco2': ds_gd['spco2'], 'tidx': tidx, 'yidx': yidx, 'xidx': xidx})
      .dropna()  # Drop rows where binning failed
      .groupby(['tidx', 'yidx', 'xidx'])
      .agg(['mean', 'std'])
      .reset_index()
      .rename(columns={'tidx': 'it', 'yidx': 'iy', 'xidx': 'ix'}))
'''
df = (pd.Series(ds_gd.spco2)
          .groupby([tidx, yidx, xidx])
          .agg(['mean', 'std'])
          .reset_index()
          .rename(columns={'level_0': 'it',
                           'level_1': 'iy',
                           'level_2': 'ix'}))
'''
df = df.dropna()

In [ ]:
df

In [ ]:
# Convert indices to integers
it, iy, ix = df[['it', 'iy', 'ix']].values.T.astype(int)

# Create DataArray to store the results
xda_mean = xda.copy() * np.nan
xda_std = xda.copy() * np.nan
xda_mean.name = 'spco2_mean'
xda_std.name = 'spco2_std'

# Place the binned mean and standard deviation into the DataArray
xda_mean.values[it, iy, ix] = df[('spco2', 'mean')]
xda_std.values[it, iy, ix] = df[('spco2', 'std')]

# Merge into a single Dataset
out = xr.merge([xda_mean, xda_std])

# Rename variables
glodap_out = out.rename({'spco2_mean': 'spco2_mean', 'spco2_std': 'spco2_std'})

In [ ]:
# Save to GCS as zarr

# Detect the year of GLODAP file name
filename = gcs_path.rsplit('/', 1)[-1]
match = re.search(r'\d{4}', filename)
if match:
    updated_year = match.group()
else:
    print("Fail to detect the year info from the GLODAP file name!")

# Set the save path and name of processed netCDF file
zarr_gcs_path = f'{save_path}/processed/GLODAPv2_spco2_{updated_year}.zarr'

glodap_out.to_zarr( zarr_gcs_path, mode='w')

# Check if saved sucessfully
if fs.exists(zarr_gcs_path):
    print(f"Successfully saved to {zarr_gcs_path}")
else:
    print(f"Failed to save to {zarr_gcs_path}")

In [ ]:
fs.ls(save_path + '/processed')

In [ ]:
glodap_out.spco2_mean.sel(time = '2021-05').plot()

In [ ]:
# If you want to delete file from the GCS path, run the following code
'''
delete_path = 'YOUR_GCS_PATH/Taylor_data/GLODAPv2.2023_Merged_Master_File.csv'

fs = gcsfs.GCSFileSystem()

if fs.exists(delete_path):
    fs.rm(delete_path, recursive=True)
    print(f"The file {delete_path} has been deleted")
else:
    print(f"The file {delete_path} does not exist")
'''